# Interactive notebooks

`ipywidgets` provides the Python bindings for interactive elements in Jupyter notebooks.  Bindings for other language to use with non-Python kernels are available as well.

In [ ]:
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import warnings

## Fast compute

### Example 1: sigmoid function

Define a function that will plot $\tanh(\beta x)$ for $x \in [-5, 5]$ and $\beta > 0$ a parameter value.

In [ ]:
def plot_tanh(beta):
    x = np.linspace(-5.0, 5.0, 101)
    y = np.tanh(beta*x)
    _ = plt.plot(x, y)

Now this function can be run for various values of `beta`, e.g.,

In [ ]:
plot_tanh(0.6)

In [ ]:
plot_tanh(4.0)

However, it would be much more interesting if the value of `beta` could be modified interactively, the plot modified on the fly.  A simple function decorated with accomplish this easily.

In [ ]:
@interact(beta=(0.2, 5.0, 0.2))
def plot_tanh(beta):
    x = np.linspace(-5.0, 5.0, 101)
    y = np.tanh(beta*x)
    _ = plt.plot(x, y)

A plot can be parameterized by multiple values, either numerical or categorical.

### Example 2: viral load

A model for the viral load is given by
$$
V(t) = A e^{-\alpha t} + B e^{-\beta t}
$$
This expression can be rewritten as
$$
V(t) = A e^{-\alpha t} (1 + \frac{B}{A} e^{-(\beta - \alpha)t}
$$
In order to study this function qualitatively, we can set $A = 1$ and $\alpha = 1$.  We know that $-1 \leq B \leq 0$, $1 \leq \beta$, two independent quantities.

In [ ]:
@interact(B=(-1.0, 0.0, 0.05), beta=(1.0, 8.0, 0.1))
def viral_load_plot(B, beta):
    t = np.linspace(0.0, 7.0, 101)
    v = np.exp(-t)*(1.0 + B*np.exp(-(beta - 1.0)*t))
    _ = plt.plot(t, v)
    _ = plt.ylim(0.0, 1.0)
    _ = plt.xlabel('$t$')
    _ = plt.ylabel('$V(t)$')

## Slow compute

Define a function that computes the number of iterations of $z = z^2 + c$ such that $|z| < 2$ in the complex plane.

In [ ]:
def compute_fractal(c_re, c_im):
    c = complex(c_re, c_im)
    max_iters = 255
    nr_points = 300
    max_val = 1.8
    max_norm = 2.0
    x = np.linspace(-max_val, max_val, nr_points)
    y = np.linspace(-max_val, max_val, nr_points)
    X, Y = np.meshgrid(x, y)
    Z = X + Y*1j
    iterations = np.zeros(Z.shape, dtype=np.uint8)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for _ in range(max_iters):
            Z = Z**2 + c
            iterations[np.abs(Z) < max_norm] += 1
    return iterations
        

Define a function to plot the result of that function as a heatmap.

In [ ]:
def plot_fractal(c_re, c_im):
    ns = compute_fractal(c_re, c_im)
    _, axes = plt.subplots()
    axes.imshow(ns)
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)

You can call this function for various values of the real and imaginary part of $c$.

In [ ]:
plot_fractal(-0.622772, 0.52193j)

However, the function takes a while to evaluate.

In [ ]:
%timeit compute_fractal(-0.6, 0.4)

On average, it takes more than half a second to complete the computation, so making this interactive and just touching the sliders would result in jaggy output (at least).  Hence `interact_manual` is more appropriate, since the computation is only initiated when the `Run interact` button is pressed.

In [ ]:
_ = interact_manual(plot_fractal, c_re=(-1.0, 1.0, 0.01), c_im=(-1.0, 1.0, 0.01))